In [ ]:
import pandas as pd
import csv
from mysql.connector import connect

FILE_PATH = "chatgpt1.csv"
df = pd.read_csv(FILE_PATH)
df.head()

In [ ]:
# Changing datetime column to SQL format datetime
df['Datetime'] = df['Datetime'].apply(lambda x: x.split("+")[0])

In [ ]:
# To extract value from Source field by eliminating html tags

from bs4 import BeautifulSoup

def extract_source(html_tag):
    tag_info = BeautifulSoup(html_tag)
    tag = tag_info.find('a')
    source = tag.contents[0]
    return source

In [ ]:
df['Source'] = df['Source'].apply(extract_source)

In [ ]:
# Renaming columns

df.columns = ['tweet_time', 'tweet_id', 'Text', 'Username', 'Permalink', 'User', 
              'Outlinks', 'CountLinks', 'ReplyCount', 'RetweetCount', 'LikeCount',
              'QuoteCount', 'ConversationId', 'Language', 'Source', 'Media','QuotedTweet', 
              'MentionedUsers', 'hashtag', 'hastag_counts']

In [ ]:
# Changing all hashtags to lower case to avoid duplicants

def lower_hashtags(tags_str):
    tags_list = eval(tags_str)
    tags = [tag.lower() for tag in tags_list]
    return tags
    
df['hashtag'] = df['hashtag'].apply(lower_hashtags)

In [ ]:
# Creating hashtags table (hashtag_id, hashtag)

hashtag_dict = {}

hashtag_list = set()
for row in df.itertuples():
    if row.hastag_counts > 0:
        for tag in row.hashtag:
            hashtag_list.add(tag)

file = open("hashtags_table.csv", "w", encoding='UTF-16', newline='')
writer = csv.DictWriter(file, fieldnames=["id", "hashtag"])
i = 1
writer.writeheader()
for row in hashtag_list:
    hashtag_dict[row] = i
    writer.writerow({"id":i, "hashtag":row})
    i += 1

file.close()

In [ ]:
# creating tweets has hashtags table

file = open("tweets_has_hashtags_table.csv", "w", encoding='UTF-16', newline='')
writer = csv.DictWriter(file, fieldnames=["tweet_id", "hashtag_id"])
writer.writeheader()
for row in df.itertuples():
    if row.hastag_counts > 0:
        for tag in row.hashtag:
            writer.writerow({"tweet_id": row.tweet_id, "hashtag_id": hashtag_dict[tag]})

file.close()

In [ ]:
# Creating tweet metrics table

df1 = df[["tweet_id", "LikeCount", "RetweetCount", "ReplyCount", "QuoteCount", "hastag_counts", 
          "Language", "Source", "MentionedUsers", "Media", "Outlinks", "CountLinks"]]

df1.to_csv("tweet_metrics_table.csv", index=False)

In [ ]:
# Creating users table

users = set()
for row in df.itertuples():
    users.add(row.Username)

file = open("users_table.csv", "w", newline='')
writer = csv.DictWriter(file, fieldnames=["user_id", "username"])
writer.writeheader()

uid = 1
for row in users:
    writer.writerow({"user_id": uid, "username":row})
    uid += 1

file.close()

In [ ]:
# Adding user_id column to main DataFrame

ucount = 1
user_dict = {}
for user in users:
    user_dict[user] = ucount
    ucount += 1

df["user_id"] = df["Username"].apply(lambda x: user_dict[x])

In [ ]:
# Creating tweets table

tweets = df[["tweet_id", "Text", "tweet_time", "user_id", "Permalink", "ConversationId", "QuotedTweet"]]
tweets.to_csv("tweets_table.csv", index=False)

In [ ]:
# Establishing Connection to Database (host for Cloud Connection -> RDS Instance EndPoint)
# Enter DB username, password, schema name

from mysql.connector import connect

db_connection = connect(host="database-lab1.c4j8ssgcqmdi.us-west-1.rds.amazonaws.com", user=input("Enter username: "), password=getpass("Enter password: "), database=input("Enter the Schema Name: "))
cursor = db_connection.cursor()

We will now execute SQL code to define tables and relationships and later insert the data into tables

In [ ]:
CREATE_USERS_TABLE = "CREATE TABLE users(user_id INT AUTO_INCREMENT PRIMARY KEY, username VARCHAR(255), post_count INT DEFAULT 0)"

In [ ]:
cursor.execute(CREATE_USERS_TABLE)
db_connection.commit()

In [ ]:
CREATE_TWEETS_TABLE = "CREATE TABLE tweets(tweet_id BIGINT PRIMARY KEY, tweet_text TEXT(5000), tweet_time DATETIME, user_id INT, Permalink VARCHAR(255), ConversationId BIGINT, QuotedTweet VARCHAR(255), FOREIGN KEY(user_id) REFERENCES users(user_id))"

In [ ]:
cursor.execute(CREATE_TWEETS_TABLE)
db_connection.commit()

#### Trigger to increment post_count if a new tweet is inserted in tweet table
##### Executed in workbench

In [ ]:
CREATE_HASHTAGS_TABLE = "CREATE TABLE hashtags(hashtag_id INT PRIMARY KEY, hashtag VARCHAR(255))"

In [ ]:
cursor.execute(CREATE_HASHTAGS_TABLE)
db_connection.commit()

In [ ]:
CREATE_THH_TABLE = "CREATE TABLE tweets_has_hashtags (tweet_id BIGINT, hashtag_id INT, FOREIGN KEY(tweet_id) REFERENCES tweets(tweet_id), FOREIGN KEY(hashtag_id) REFERENCES hashtags(hashtag_id))"

In [ ]:
cursor.execute(CREATE_THH_TABLE)
db_connection.commit()

In [ ]:
CREATE_TWEET_METRICS_TABLE = "CREATE TABLE tweet_metrics(tweet_id BIGINT, like_count INT, retweet_count INT, reply_count INT, \
                                quote_count INT, hashtag_count INT, language VARCHAR(20), source VARCHAR(45), \
                                mentioned_users TEXT(5000), media TEXT(5000), outlinks TEXT(5000), countlinks TEXT(5000), \
                                FOREIGN KEY(tweet_id) REFERENCES tweets(tweet_id))"

In [ ]:
cursor.execute(CREATE_TWEET_METRICS_TABLE)
db_connection.commit()

In [ ]:
# Insert values into users table

with open('users_table.csv') as csvfile:
    csvreader = csv.reader(csvfile)
    
    # To skip the header row (title)
    header = next(csvreader)

    for row in csvreader:
        sql = "INSERT INTO users (username) VALUES ('{}')".format(row[1])
        cursor.execute(sql)

# Save changes to database
db_connection.commit()

In [ ]:
# Insert values into tweets table

with open('tweets_table.csv', "r", encoding='utf-8', errors='ignore') as csvfile:
    csvreader = csv.reader(csvfile)
    header = next(csvreader)
    
    for row in csvreader:
        sql = """INSERT INTO tweets VALUES ({}, %s, '{}', {}, %s, {}, '{}')""".format(row[0], row[2], row[3], row[5], row[6])
        cursor.execute(sql, tuple([row[1], row[4]]))

db_connection.commit()

In [ ]:
# Insert values into hashtags table

with open('hashtags_table.csv', "r", encoding='utf-16', errors='ignore') as csvfile:
    csvreader = csv.reader(csvfile)
    header = next(csvreader)
    
    for row in csvreader:
        sql = """INSERT INTO hashtags VALUES ({}, %s)""".format(row[0])  
        cursor.execute(sql, tuple([row[1]]))

db_connection.commit()

In [ ]:
# Insert values into tweets_has_hashtags table

with open('tweets_has_hashtags_table.csv', "r", encoding='utf-16', errors='ignore') as csvfile:
    csvreader = csv.reader(csvfile)
    header = next(csvreader)
    
    for row in csvreader:
        sql = """INSERT INTO tweets_has_hashtags VALUES ({}, {})""".format(row[0], row[1])
        cursor.execute(sql)

db_connection.commit()

In [ ]:
# Insert values into tweet_metrics table

with open('tweet_metrics_table.csv', "r", encoding='utf-8', errors='ignore') as csvfile:
    csvreader = csv.reader(csvfile)
    header = next(csvreader)
    
    for row in csvreader:
        sql = """INSERT INTO tweet_metrics VALUES ({}, {}, {}, {}, {}, {}, %s, %s, %s, %s, %s, %s)""".format(int(row[0]), int(row[1]), 
                                                                                                             int(row[2]), int(row[3]), 
                                                                                                             int(row[4]), int(row[5]))
        cursor.execute(sql, tuple([row[6], row[7], row[8], row[9], row[10], row[11]]))

db_connection.commit()

In [ ]:
cursor.close()
db_connection.close()

In [1]:
# Demonstrating AWS Connectivity with python

from mysql.connector import connect

endpoint = "database-lab1.c4j8ssgcqmdi.us-west-1.rds.amazonaws.com"
try:
    db_connection = connect(host=endpoint, username = input("Enter username: "), password=input("Enter password: "), database="chatgpt")
except Exception as e:
    print("Error Occurred - {}. Please try again".format(e))
    
cursor = db_connection.cursor()
query = "SELECT u.username, AVG(tm.like_count) FROM tweet_metrics tm JOIN tweets t ON tm.tweet_id = t.tweet_id JOIN users u ON t.user_id = u.user_id GROUP BY u.username ORDER BY AVG(tm.like_count) DESC"

cursor.execute(query)
results = cursor.fetchall()

Enter username: keerthana
Enter password: notsostrong


In [3]:
print("Top 5 users with highest average like counts..")
for i in range(5):
    print(results[i])

Top 5 users with highest average like counts..
('GRDecter', Decimal('24061.0000'))
('WatcherGuru', Decimal('10836.5000'))
('Veskii_', Decimal('9125.0000'))
('mccormick_ted', Decimal('8468.5000'))
('sashishi_EN', Decimal('6230.6667'))


In [4]:
cursor.close()
db_connection.close()